In [1]:
 import numpy as np
import pandas as pd
import matplotlib as mpl
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator as gen
from sklearn.preprocessing import MinMaxScaler, StandardScaler

 

,new_deaths,positive_rate
count,500.000000,500.000000
mean,1.646000,0.068814
std,3.354876,0.061634
min,0.000000,0.000000
25%,0.000000,0.023000
50%,0.000000,0.045000
75%,2.000000,0.100000
max,28.000000,0.319000


In [2]:
dataset['date']=pd.to_datetime(dataset['date'], infer_datetime_format= True)
df= dataset.loc[:,dataset.columns !='date']

In [3]:
scaler= MinMaxScaler()

dataScaled=scaler.fit_transform(df)

In [4]:
features= dataScaled
target= dataScaled[:,0]

In [5]:
X_train, X_test, Y_train, Y_test = train_test_split(features, target, test_size= 0.10, random_state=123, shuffle=False) 

In [6]:
X_train.shape

(450, 2)

In [7]:
win_length= 9
batch_size= 1
num_features= 2
train_generator= gen(X_train,Y_train,length=win_length,sampling_rate=1, batch_size= batch_size)
test_generator= gen(X_test,Y_test,length=win_length,sampling_rate=1, batch_size= batch_size)

In [8]:
train_generator[0]

(array([[[0.        , 0.        ],
         [0.        , 0.        ],
         [0.        , 0.        ],
         [0.        , 0.        ],
         [0.        , 0.        ],
         [0.        , 0.        ],
         [0.        , 0.        ],
         [0.03571429, 0.        ],
         [0.        , 0.        ]]]),
 array([0.03571429]))

In [9]:
model= tf.keras.Sequential()
model.add(tf.keras.layers.LSTM(128,input_shape=(win_length,num_features),return_sequences=True))
model.add(tf.keras.layers.LeakyReLU(alpha=0.5))
model.add(tf.keras.layers.LSTM(128,return_sequences=True))
model.add(tf.keras.layers.LeakyReLU(alpha=0.5))
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.LSTM(64,return_sequences=False))
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Dense(1))

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 9, 128)            67072     
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 9, 128)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 9, 128)            131584    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 9, 128)            0         
_________________________________________________________________
dropout (Dropout)            (None, 9, 128)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0

In [11]:
early_stopping= tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=2,mode='min')
model.compile(loss=tf.losses.MeanSquaredError(),optimizer=tf.optimizers.Adam(),metrics=[tf.metrics.MeanAbsoluteError()])
history=model.fit(train_generator,epochs=50,validation_data=test_generator,shuffle=False, callbacks=[early_stopping])


Epoch 1/50
441/441 [==============================] - 4s 9ms/step - loss: 0.0131 - mean_absolute_error: 0.0720 - val_loss: 0.0021 - val_mean_absolute_error: 0.0345
Epoch 2/50
441/441 [==============================] - 3s 7ms/step - loss: 0.0140 - mean_absolute_error: 0.0746 - val_loss: 0.0021 - val_mean_absolute_error: 0.0342
Epoch 3/50
441/441 [==============================] - 3s 7ms/step - loss: 0.0138 - mean_absolute_error: 0.0761 - val_loss: 0.0022 - val_mean_absolute_error: 0.0326


In [12]:
model.evaluate_generator(test_generator, verbose=0)

Instructions for updating:
Please use Model.evaluate, which supports generators.


[0.0021617719903588295, 0.032570160925388336]

In [13]:
predictions=model.predict_generator(test_generator)

Instructions for updating:
Please use Model.predict, which supports generators.


In [14]:
predictions.shape[0]

41

In [15]:
predictions

array([[0.01350585],
       [0.01355905],
       [0.01371242],
       [0.01369872],
       [0.01374606],
       [0.01387627],
       [0.01402533],
       [0.01423327],
       [0.01443554],
       [0.01445935],
       [0.01448725],
       [0.01445687],
       [0.01426499],
       [0.01415347],
       [0.014046  ],
       [0.01389837],
       [0.01390615],
       [0.01408071],
       [0.01432901],
       [0.01471967],
       [0.0151892 ],
       [0.0155493 ],
       [0.01589791],
       [0.01623392],
       [0.01638068],
       [0.01660237],
       [0.01681868],
       [0.01696322],
       [0.01710203],
       [0.01727613],
       [0.01748839],
       [0.01755366],
       [0.01745682],
       [0.01733844],
       [0.01695213],
       [0.01628185],
       [0.01554963],
       [0.01463966],
       [0.01388428],
       [0.01289307],
       [0.01212333]], dtype=float32)

In [16]:
Y_test

array([0.        , 0.        , 0.07142857, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.03571429, 0.        ,
       0.        , 0.10714286, 0.        , 0.        , 0.03571429,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.03571429, 0.        , 0.        , 0.03571429, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.03571429, 0.        , 0.        , 0.07142857, 0.07142857,
       0.        , 0.03571429, 0.        , 0.14285714, 0.        ,
       0.14285714, 0.07142857, 0.        , 0.10714286, 0.        ,
       0.        , 0.14285714, 0.        , 0.07142857, 0.07142857])

In [17]:
X_test[:,1:][win_length:]

array([[0.05642633],
       [0.05642633],
       [0.05642633],
       [0.05329154],
       [0.05329154],
       [0.06896552],
       [0.06269592],
       [0.04702194],
       [0.05015674],
       [0.04075235],
       [0.03761755],
       [0.05015674],
       [0.04702194],
       [0.04388715],
       [0.07523511],
       [0.06896552],
       [0.06896552],
       [0.0846395 ],
       [0.10031348],
       [0.09717868],
       [0.0815047 ],
       [0.0815047 ],
       [0.10971787],
       [0.10658307],
       [0.10658307],
       [0.12225705],
       [0.11285266],
       [0.12539185],
       [0.09404389],
       [0.10344828],
       [0.09717868],
       [0.10971787],
       [0.10344828],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ]])

In [18]:
df_pred=pd.concat([pd.DataFrame(predictions), pd.DataFrame(X_test[:,1:][win_length:])], axis=1)

In [19]:
Y_test

array([0.        , 0.        , 0.07142857, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.03571429, 0.        ,
       0.        , 0.10714286, 0.        , 0.        , 0.03571429,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.03571429, 0.        , 0.        , 0.03571429, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.03571429, 0.        , 0.        , 0.07142857, 0.07142857,
       0.        , 0.03571429, 0.        , 0.14285714, 0.        ,
       0.14285714, 0.07142857, 0.        , 0.10714286, 0.        ,
       0.        , 0.14285714, 0.        , 0.07142857, 0.07142857])

In [20]:
df_pred= pd.concat([pd.DataFrame(predictions), pd.DataFrame(X_test[:,1:][win_length:])], axis=1)

In [21]:
rev_trans= scaler.inverse_transform(df_pred)

In [22]:
rev_trans

array([[0.3781638 , 0.018     ],
       [0.37965342, 0.018     ],
       [0.38394779, 0.018     ],
       [0.38356414, 0.017     ],
       [0.38488969, 0.017     ],
       [0.38853557, 0.022     ],
       [0.39270915, 0.02      ],
       [0.39853163, 0.015     ],
       [0.40419525, 0.016     ],
       [0.40486167, 0.013     ],
       [0.40564294, 0.012     ],
       [0.40479241, 0.016     ],
       [0.39941971, 0.015     ],
       [0.39629704, 0.014     ],
       [0.39328806, 0.024     ],
       [0.38915423, 0.022     ],
       [0.38937213, 0.022     ],
       [0.39426   , 0.027     ],
       [0.40121235, 0.032     ],
       [0.41215083, 0.031     ],
       [0.42529762, 0.026     ],
       [0.43538035, 0.026     ],
       [0.44514161, 0.035     ],
       [0.45454979, 0.034     ],
       [0.4586589 , 0.034     ],
       [0.46486628, 0.039     ],
       [0.47092293, 0.036     ],
       [0.47497009, 0.04      ],
       [0.47885671, 0.03      ],
       [0.48373155, 0.033     ],
       [0.

In [23]:
df_final = df[predictions.shape[0]*-1:]

In [24]:
df_final.count()

new_deaths       41
positive_rate    41
dtype: int64

In [25]:
df_final['new_deaths_pred'] = rev_trans[:,0]

<ipython-input-25-558a1a35d4a0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['new_deaths_pred'] = rev_trans[:,0]


In [26]:
df_final

,new_deaths,positive_rate,new_deaths_pred
459,0.0,0.018,0.378164
460,0.0,0.018,0.379653
461,3.0,0.018,0.383948
462,0.0,0.017,0.383564
463,0.0,0.017,0.384890
464,1.0,0.022,0.388536
465,0.0,0.020,0.392709
466,0.0,0.015,0.398532
467,0.0,0.016,0.404195
468,0.0,0.013,0.404862


In [27]:
from sklearn.metrics import mean_absolute_error
mae_new_deaths=mean_absolute_error(df_final['new_deaths'],df_final['new_deaths_pred'])
print('Mean absolute error of new deaths is:',mae_new_deaths)

Mean absolute error of new deaths is: 0.9119645182679339


In [28]:
import math 
from statistics import mean
mean_new_deaths = mean(df_final['new_deaths'])
print(mean_new_deaths)

0.8048780487804879
